# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Dani Adrian

NIM   : 225150201111009

Kelas : PPM E

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-02 10:43:50--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.2’

iris.csv.2          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-02 10:43:50 (45.9 MB/s) - ‘iris.csv.2’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [ ]:
label_uji = data_uji.pop('variety')

In [ ]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
48,5.3,3.7,1.5,0.2
77,6.7,3.0,5.0,1.7
111,6.4,2.7,5.3,1.9
8,4.4,2.9,1.4,0.2
62,6.0,2.2,4.0,1.0


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data.

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [ ]:
from collections import Counter

def hitung_prior(list_kelas):
    n_data = len(list_kelas)
    prior = Counter(list_kelas)
    for key in prior.keys():
        prior[key] = prior[key] / n_data
    return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [ ]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Setosa': 0.3333333333333333, 'Virginica': 0.3333333333333333, 'Versicolor': 0.3333333333333333})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [ ]:
def hitung_rata2_std_kelas(input_data):
    list_columns = input_data.columns[:-1]
    class_column_name = input_data.columns[-1]
    list_class = set(input_data[class_column_name])
    rata2 = {}
    std = {}
    for column in list_columns:
        for a_class in list_class:
            rata2[(a_class, column)] = input_data.loc[input_data[class_column_name] == a_class][column].mean()
            std[(a_class, column)] = input_data.loc[input_data[class_column_name] == a_class][column].std()
    return rata2, std

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [ ]:
import math

def hitung_likelihood_gaussian(data, rata2, std):
    hasil = (1 / (math.sqrt(2 * math.pi * (std ** 2)))) * math.exp((-1 * ((data - rata2) ** 2)) / (2 * (std ** 2)))
    return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas.

In [ ]:
def training_naive_bayes_gaussian(data_latih):
    class_column_name = data_latih.columns[-1]
    prior = hitung_prior(data_latih[class_column_name])
    (rata2, std) = hitung_rata2_std_kelas(data_latih)
    list_class = set(data_latih[class_column_name])
    list_columns = data_latih.columns[:-1]

    model = {}
    model['prior'] = prior
    model['rata2'] = rata2
    model['std'] = std
    model['list_class'] = list_class
    model['list_columns'] = list_columns

    return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [ ]:
model = training_naive_bayes_gaussian(data_latih)
print(model)

{'prior': Counter({'Setosa': 0.3333333333333333, 'Virginica': 0.3333333333333333, 'Versicolor': 0.3333333333333333}), 'rata2': {('Versicolor', 'sepal.length'): 5.9350000000000005, ('Setosa', 'sepal.length'): 5.0175, ('Virginica', 'sepal.length'): 6.6375, ('Versicolor', 'sepal.width'): 2.8125, ('Setosa', 'sepal.width'): 3.4200000000000004, ('Virginica', 'sepal.width'): 2.9824999999999995, ('Versicolor', 'petal.length'): 4.2299999999999995, ('Setosa', 'petal.length'): 1.465, ('Virginica', 'petal.length'): 5.577499999999999, ('Versicolor', 'petal.width'): 1.3275, ('Setosa', 'petal.width'): 0.24500000000000002, ('Virginica', 'petal.width'): 2.02}, 'std': {('Versicolor', 'sepal.length'): 0.5235725404642657, ('Setosa', 'sepal.length'): 0.36366193091936366, ('Virginica', 'sepal.length'): 0.5899532793322467, ('Versicolor', 'sepal.width'): 0.3073355303331006, ('Setosa', 'sepal.width'): 0.3831047255897932, ('Virginica', 'sepal.width'): 0.34185579505894675, ('Versicolor', 'petal.length'): 0.45018

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [ ]:
def testing_naive_bayes_gaussian(model, data_uji):
    prior = model['prior']
    rata2 = model['rata2']
    std = model['std']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = dict.fromkeys(list_class, 1)

    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] *= hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
        posterior[a_class] *= prior[a_class]

    kelas_uji = max(posterior, key=posterior.get)
    return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [ ]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model, data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Setosa
Setosa


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [ ]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
    prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))

total_benar = sum(prediksi_total[i] == label_uji.iloc[i] for i in range(len(prediksi_total)))
print("Total prediksi benar: ", total_benar)

Total prediksi benar:  28


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print('File berhasil diupload:', filename)

Saving car_sample.csv to car_sample (2).csv
File berhasil diupload: car_sample (2).csv


In [ ]:
import pandas as pd
data_tugas = pd.read_csv('car_sample.csv')
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [ ]:
tugas_latih,tugas_uji = train_test_split(data_tugas,test_size=0.3)
print(tugas_latih.shape[0])
print(tugas_uji.shape[0])

1209
519


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [ ]:
tugas_label_uji = tugas_uji.pop('class')
tugas_uji.head()

,buying,maint,lug_boot,safety
1666,low,low,small,med
189,vhigh,high,small,low
276,vhigh,med,big,low
133,vhigh,high,big,med
756,high,low,small,low


Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [ ]:
prior_tugas = hitung_prior(tugas_latih['class'])
print(prior_tugas)

Counter({'unacc': 0.696443341604632, 'acc': 0.22332506203473945, 'good': 0.04218362282878412, 'vgood': 0.0380479735318445})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [ ]:
def likelihood_count(data_latih, a_class, column, values):
  column_values = data_latih[column] == values
  class_values = data_latih['class'] == a_class
  hasil = len(data_latih[(column_values)&(class_values)])+1/(data_latih[column].count())
  return hasil

In [ ]:
def hitung_likelihood_multinomial(data_latih):
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(data_latih[class_column_name])
  likelihood = {}
  for column in list_columns:
    for val in data_latih[column].unique():
      for a_class in list_class:
        likelihood[(column, val, a_class)] = (likelihood_count(data_latih, a_class, column, val))
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [ ]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'high', 'acc'): 76.00082712985939, ('buying', 'high', 'good'): 0.0008271298593879239, ('buying', 'high', 'unacc'): 215.00082712985937, ('buying', 'high', 'vgood'): 0.0008271298593879239, ('buying', 'med', 'acc'): 86.00082712985939, ('buying', 'med', 'good'): 18.00082712985939, ('buying', 'med', 'unacc'): 191.00082712985937, ('buying', 'med', 'vgood'): 18.00082712985939, ('buying', 'low', 'acc'): 57.00082712985939, ('buying', 'low', 'good'): 33.00082712985939, ('buying', 'low', 'unacc'): 180.00082712985937, ('buying', 'low', 'vgood'): 28.00082712985939, ('buying', 'vhigh', 'acc'): 51.00082712985939, ('buying', 'vhigh', 'good'): 0.0008271298593879239, ('buying', 'vhigh', 'unacc'): 256.0008271298594, ('buying', 'vhigh', 'vgood'): 0.0008271298593879239, ('maint', 'vhigh', 'acc'): 52.00082712985939, ('maint', 'vhigh', 'good'): 0.0008271298593879239, ('maint', 'vhigh', 'unacc'): 247.00082712985937, ('maint', 'vhigh', 'vgood'): 0.0008271298593879239, ('maint', 'low', 'acc'): 69.00

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [ ]:
def training_naive_bayes_multinomial(tugas_latih):
    class_column_name = tugas_latih.columns[-1]
    list_class = set(tugas_latih[class_column_name])
    list_columns = tugas_latih.columns[:-1]
    prior = {}
    likelihood = {}

    # Hitung nilai prior
    total_samples = len(tugas_latih)
    for a_class in list_class:
        prior[a_class] = len(tugas_latih[tugas_latih[class_column_name] == a_class]) / total_samples

    # Hitung nilai likelihood
    for a_class in list_class:
        likelihood[a_class] = {}
        for column in list_columns:
            likelihood[a_class][column] = {}
            for value in tugas_latih[column].unique():
                likelihood[a_class][column][value] = (len(tugas_latih[(tugas_latih[class_column_name] == a_class) & (tugas_latih[column] == value)]) + 1) / (len(tugas_latih[tugas_latih[class_column_name] == a_class]) + len(tugas_latih[column].unique()))

    model = {}
    model['prior'] = prior
    model['likelihood'] = likelihood
    model['list_class'] = list_class
    model['list_columns'] = list_columns
    return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [ ]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [ ]:
import pandas as pd

def testing_naive_bayes_multinomial(model, tugas_uji):
    prior = model['prior']
    likelihood = model['likelihood']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = {a_class: 1 for a_class in list_class}
    tugas_uji_df = pd.DataFrame([tugas_uji], columns=list_columns)  # Konversi array NumPy menjadi DataFrame
    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] *= likelihood[a_class][column][tugas_uji_df[column].iloc[0]]  # Akses nilai kolom menggunakan iloc[0]
        posterior[a_class] *= prior[a_class]
    kelas_uji = max(posterior, key=posterior.get)
    return kelas_uji


Lakukan pengujian menggunakan sebuah data uji

In [ ]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas, uji)
print("Kelas prediksi:", kelas_uji)
print("Kelas sebenarnya:", tugas_label_uji.iloc[idx])

Kelas prediksi: unacc
Kelas sebenarnya: acc


Lakukan pengujian terhadap semua data uji.

In [ ]:
for idx, uji in enumerate(tugas_uji.values):
    kelas_uji = testing_naive_bayes_multinomial(model_tugas, uji)
    print("Data uji ke-", idx, "- Kelas prediksi:", kelas_uji, "- Kelas sebenarnya:", tugas_label_uji.iloc[idx])

Data uji ke- 0 - Kelas prediksi: unacc - Kelas sebenarnya: acc
Data uji ke- 1 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 2 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 3 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 4 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 5 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 6 - Kelas prediksi: unacc - Kelas sebenarnya: good
Data uji ke- 7 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 8 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 9 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 10 - Kelas prediksi: unacc - Kelas sebenarnya: acc
Data uji ke- 11 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 12 - Kelas prediksi: acc - Kelas sebenarnya: unacc
Data uji ke- 13 - Kelas prediksi: unacc - Kelas sebenarnya: unacc
Data uji ke- 14 - Kelas prediksi: unacc - Kelas sebenarnya: acc
Data uji ke- 15 - Kelas predi

In [ ]:
len(tugas_label_uji)

519